# 03_fredf_baseline

Прототип реализации FreDF и сравнение с базовыми моделями.

# Применение моделей на исходных данных

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.dpi'] = 300

from ts_toolkit.io import clean_timeseries
from ts_toolkit.calendar import add_hour_sin_cos
from ts_toolkit.viz import plot_history_forecast
from ts_toolkit.split import three_way_split
from src.models.catboost_delay_model import DelayForecastModel
from ts_toolkit.metrics import daily_mae
from ts_toolkit.metrics import global_metrics

In [2]:
from src.data_loader import fetch_frame
# df = fetch_frame()
df = fetch_frame(
    use_cache=True,
    cache_filename="only_common_delayp90.parquet"
)
print(df.head())

print(df.info())

print(df.describe())

print(df.isnull().sum())


                     common_delay_p90
ts                                   
2025-04-27 18:00:30       2394.210526
2025-04-27 18:00:45       2398.424069
2025-04-27 18:01:00       2396.124524
2025-04-27 18:01:15       2417.008604
2025-04-27 18:01:30       2420.737327
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90643 entries, 2025-04-27 18:00:30 to 2025-05-13 11:41:00
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   common_delay_p90  90641 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB
None
       common_delay_p90
count      90641.000000
mean        1802.349510
std          901.617551
min          235.000000
25%          922.395833
50%         2094.852941
75%         2382.755633
max         8500.000000
common_delay_p90    2
dtype: int64


/home/borovets.av/kolya/realtime_anomaly_analytics/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df.info()

df.describe()

df.isnull().sum()

df.head()

df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90643 entries, 2025-04-27 18:00:30 to 2025-05-13 11:41:00
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   common_delay_p90  90641 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB


,common_delay_p90
ts,
2025-05-13 11:40:00,3738.866397
2025-05-13 11:40:15,3325.806452
2025-05-13 11:40:30,2479.835391
2025-05-13 11:40:45,2452.444134
2025-05-13 11:41:00,2498.843188


In [4]:
df = clean_timeseries(df, 'common_delay_p90')   # вместо блока 0‑3
# df = add_hour_sin_cos(df)
# feature_cols = ['hour_sin', 'hour_cos']

In [ ]:
# feature_cols

In [ ]:
# инициализируем — можно менять lags/roll_windows/test_size
model = DelayForecastModel(
    horizon       = 5760,   # сутки = 24*60*60 / 15 = 5760 точек
    test_size     = 0.2,    # 80 % train, 20 % hold-out
    lags          = [1, 2, 4, 96, 192, 5760],
    # roll_windows  = [96, 384, 5760]          # 24 мин, 1 ч, 1 сут
    roll_windows  = [4,96,192,1920,2880,4320,5760,8640]
)


# fit + автоматически нарисует график «train / test-true / test-pred»
train_df, test_df = model.fit(
    df,
    target_col   = 'common_delay_p90',
    feature_cols = feature_cols,      # если есть другие метрики — впишите их названия здесь
    plot         = True
)

In [ ]:
print(len(train_df), len(test_df))

In [ ]:
# берём последние (max(lags, roll_windows) + horizon) точек
history_needed = max(model.lags + model.roll_windows) + model.horizon
df_last = df.tail(history_needed)

# готовим фичи и предсказываем
df_future = model.prepare_future(df_last, 'common_delay_p90')

y_hat_next_day = model.predict(df_future)[-model.horizon:]
future_index   = pd.date_range(
    start=df.index[-1] + pd.Timedelta(seconds=15),
    periods=model.horizon,
    freq='15S'
)

from ts_toolkit.viz import plot_history_forecast

# ----- визуализация прогноза -----
history_series = df_last['common_delay_p90'].iloc[-3*5760:]
forecast_series = pd.Series(y_hat_next_day, index=future_index)

plot_history_forecast(
    history=history_series,
    forecast=forecast_series,
    title='p90 latency — history vs 24 h forecast'
)


## 2. Код-шаблоны для подробного анализа


In [ ]:
# ---------------------------------------------------
# 2.1  получить true & pred на тесте
# ---------------------------------------------------
#  полный список признаков, который модель видела
feat_cols = model.model.feature_names_          # ровно в том порядке!
test_feat  = test_df[feat_cols]                 # никаких пропусков
assert list(test_feat.columns) == list(model.model.feature_names_)

# 1. истинные значения и прогноз
test_true  = test_df['common_delay_p90']
test_pred  = pd.Series(
    model.model.predict(test_feat),
    index=test_true.index,
    name='pred'
)

# 2. остатки
resid = test_true - test_pred


# ---------------------------------------------------
# 2.2  сводные метрики
# ---------------------------------------------------
metrics = global_metrics(test_true, test_pred)
metrics_df = pd.DataFrame([metrics]).T.rename(columns={0: "value"})
print("\n*** Hold-out metrics ***")
print(metrics_df)

# ---------------------------------------------------
# 2.3  метрики по суткам
# ---------------------------------------------------
daily_mae_result = daily_mae(test_true, test_pred)
print("\nMAE by day:")
print(daily_mae_result.tail())

# ---------------------------------------------------
# 2.4  распределение ошибок
# ---------------------------------------------------
plt.figure(figsize=(12,4))
plt.hist(resid, bins=100, alpha=.7, edgecolor='black')
plt.axvline(resid.mean(), color='r', linestyle='--', label=f"mean={resid.mean():.1f}")
plt.title("Residual distribution on test")
plt.xlabel("error (true − pred)")
plt.legend(); plt.tight_layout(); plt.show()

# ---------------------------------------------------
# 2.5  true vs pred scatter
# ---------------------------------------------------
plt.figure(figsize=(6,6))
plt.scatter(test_true, test_pred, s=3, alpha=0.5)
lim = [0, max(test_true.max(), test_pred.max())*1.05]
plt.plot(lim, lim, 'k--')
plt.xlabel("true"); plt.ylabel("pred")
plt.title("True vs predicted, test split")
plt.tight_layout(); plt.show()


## 3. Feature Importance (какие лаги реально работают)

In [ ]:
feat_names = model.model.feature_names_        # ← то, что CatBoost запомнил
importances = model.model.get_feature_importance(type='FeatureImportance')

# в один датафрейм
imp_df = (pd.DataFrame({"feature": feat_names,
                        "importance": importances})
            .sort_values(by="importance", ascending=False)
            .reset_index(drop=True))

# ---------------------------------------------------
#  топ-20 на графике
# ---------------------------------------------------
plt.figure(figsize=(10,6))
plt.barh(imp_df.feature.head(20)[::-1],
         imp_df.importance.head(20)[::-1])
plt.title("Top-20 feature importances (CatBoost)")
plt.tight_layout(); plt.show()


In [ ]:
plt.figure(figsize=(14,4))
plt.plot(resid.index, resid, alpha=0.7)
plt.title("Residuals over time (test split)")
plt.axhline(0, color='k', lw=1)
plt.tight_layout(); plt.show()


# Проверка на реальных данных

In [ ]:
# ─────────────────────────────────────────────────────────────
# 1.  выбираем точку отсечки (24 h до конца ряда)
# ─────────────────────────────────────────────────────────────
step       = pd.Timedelta(seconds=15)
H          = model.horizon                   # 5760
cut_time   = df.index[-H-1]                  # last seen by model

hist_need  = max(model.lags + model.roll_windows) + 10
df_hist    = df.loc[:cut_time].tail(hist_need)

# ─────────────────────────────────────────────────────────────
# 2.  генерируем признаки и прогноз
# ─────────────────────────────────────────────────────────────
df_future  = model.prepare_future(df_hist, 'common_delay_p90')
y_pred     = model.model.predict(df_future)

pred_idx   = df_future.index            # ровно столько, сколько точек в y_pred
pred_ser   = pd.Series(y_pred, index=pred_idx, name='pred')

# ─────────────────────────────────────────────────────────────
# 3.  реальные значения (align по pred_idx)
# ─────────────────────────────────────────────────────────────
y_true = df['common_delay_p90'].loc[pred_idx]   # факты под теми же метками

# ─────────────────────────────────────────────────────────────
# 4.  метрики
# ─────────────────────────────────────────────────────────────
metrics = global_metrics(y_true, pred_ser)
print(f"Blind 24-h test • MAE={metrics['MAE']:.1f}  RMSE={metrics['RMSE']:.1f}  MAPE={metrics['MAPE']:.2f}%  "
      f"on {len(y_true)} valid points")

# ─────────────────────────────────────────────────────────────
# 5.  график: 3 сут истории + пред / факт
# ─────────────────────────────────────────────────────────────

hist_start = cut_time - pd.Timedelta(hours=72)
plot_history_forecast(
    history  = df.loc[hist_start:cut_time, 'common_delay_p90'],
    forecast = pred_ser,
    actual   = y_true,
    title    = 'Blind forecast vs actual — last 24 h'
)


In [ ]:
print("train_END :", train_df.index[-1])
print("cut_time  :", cut_time)
print("cut_time > train_END ?", cut_time > train_df.index[-1])


# Максимально честный тест

In [ ]:
# Разделяем данные
df_train, df_val, df_hold = three_way_split(df, train_ratio=0.8, val_ratio=0.1)

# ── Создание и обучение модели ─────────────────────────────────
model = DelayForecastModel(
    horizon=len(df_hold), # Горизонт прогноза равен размеру hold-out
    lags=[1, 2, 4, 96, 192, 5760],
    roll_windows=[4, 96, 192, 1920, 2880, 4320, 5760, 8640]
)

# Обучаем модель. fit больше не возвращает датафреймы.
print("Начинаю обучение модели...")
model.fit(
    train_df=df_train, 
    target_col='common_delay_p90',
    val_df=df_val
    # feature_cols больше не нужны, модель сама создает календарные признаки
)
print("Модель успешно обучена!")
model.save("my_delay_model_v1") 

Начинаю обучение модели...


In [ ]:
# from src.models.catboost_delay_model import DelayForecastModel

# # Загружаем модель из папки
# loaded_model = DelayForecastModel.load("my_delay_model_v1")

# # И сразу используем для прогноза
# forecast = loaded_model.predict(df_context) 
# print(forecast.head())

In [ ]:
# ── Прогноз на будущее (hold-out) ──────────────────────────
# Для предсказания нам нужна история, достаточная для создания самых "длинных" признаков
history_need = max(model.lags + model.roll_windows)

# Используем только доступную историю (train + val)
# .tail() не нужен, так как _prepare_features сам разберется
df_context = pd.concat([df_train, df_val])

# 🎯 НОВЫЙ МЕТОД ПРОГНОЗА: один вызов возвращает весь горизонт
print("Делаю прогноз...")
y_pred_series = model.predict(df_hist=df_context)
print("Прогноз готов!")

# Подготавливаем данные для сравнения
y_true = df_hold['common_delay_p90'].reindex(y_pred_series.index).dropna()
y_pred = y_pred_series.reindex(y_true.index).dropna() # Убедимся, что все выровнено

print(f"Длина прогноза: {len(y_pred)}, Длина реальных данных: {len(y_true)}")

In [ ]:
pd.Series(y_pred, index=pred_idx).plot()

In [ ]:
pd.Series(y_true, index=pred_idx).plot()


In [ ]:
# y_true и y_pred уже созданы в предыдущей ячейке

metrics = global_metrics(y_true, y_pred)
print(f"Blind 24-h test • MAE={metrics['MAE']:.1f}  RMSE={metrics['RMSE']:.1f}  MAPE={metrics['MAPE']:.2f}%  "
      f"on {len(y_true)} valid points")

# ── график: история 3 сут + прогноз vs факт ──
# hist_start = pred_idx[0] - pd.Timedelta(hours=1)
hist_start = pred_idx[0] - pd.Timedelta(minutes=30)
plot_history_forecast(
    history  = df.loc[hist_start:pred_idx[0], 'common_delay_p90'],
    forecast = pd.Series(y_pred, index=pred_idx),
    actual   = y_true,
    title    = 'Blind forecast vs actual — hold-out 24 h'
)

In [ ]:
from ts_toolkit.metrics import daily_mae

daily_mae_df = daily_mae(pd.Series(y_true, index=pred_idx), pd.Series(y_pred, index=pred_idx))
print(daily_mae_df)

In [ ]:
# last element of df_train
df_val['common_delay_p90'].tail(1)

In [12]:
# i want to make list of last element of df_val by len of df_hold

# last element of df_val
last_element_df_val = df_val['common_delay_p90'].tail(1)

# len of df_hold
len_df_hold = len(df_hold)

# make list of last element of df_val by len of df_hold
list_of_last_elements = [float(last_element_df_val.iloc[0])] * len_df_hold

# print list_of_last_elements
# list_of_last_elements

In [13]:
# i want to make list of avg elemet by 15 minutes of concat df_train and df_val

# concat df_train and df_val
df_train_val = pd.concat([df_train, df_val])

# make list of avg elemet of df_train_val
list_of_avg_elements = [float(df_train_val['common_delay_p90'].mean())] * len(df_hold)

# print list_of_avg_elements
# list_of_avg_elements

In [14]:
# i want to make list of avg elements by last 15 minutes of concat df_train and df_val by len of df_hold

# Concatenate df_train and df_val
combined_df = pd.concat([df_train, df_val])

# Get the timestamp for 15 minutes before the last entry in combined_df
last_timestamp = combined_df.index[-1]
time_window_start = last_timestamp - pd.Timedelta(minutes=15)

# Filter data for the last 15 minutes and calculate the average
last_15_min_data = combined_df.loc[time_window_start:last_timestamp, 'common_delay_p90']
avg_last_15_minutes = last_15_min_data.mean()

# Create a list of this average, repeated len_df_hold times
list_of_avg_last_15_minutes_elements = [float(avg_last_15_minutes)] * len(df_hold)

# print list_of_avg_last_15_minutes_elements
# list_of_avg_last_15_minutes_elements



In [ ]:
y_true[1]

In [ ]:
# y_true и y_pred уже созданы в предыдущей ячейке

metrics = global_metrics(y_true, list_of_last_elements)
print(f"Blind 24-h test • MAE={metrics['MAE']:.1f}  RMSE={metrics['RMSE']:.1f}  MAPE={metrics['MAPE']:.2f}%  "
      f"on {len(y_true)} valid points")

# ── график: история 3 сут + прогноз vs факт ──
# hist_start = pred_idx[0] - pd.Timedelta(hours=1)
hist_start = pred_idx[0] - pd.Timedelta(minutes=30)
plot_history_forecast(
    history  = df.loc[hist_start:pred_idx[0], 'common_delay_p90'],
    forecast = pd.Series(list_of_last_elements, index=pred_idx),
    actual   = y_true,
    title    = 'Blind forecast vs actual — hold-out 24 h'
)

In [ ]:
# y_true и y_pred уже созданы в предыдущей ячейке

metrics = global_metrics(y_true, list_of_avg_elements)
print(f"Blind 24-h test • MAE={metrics['MAE']:.1f}  RMSE={metrics['RMSE']:.1f}  MAPE={metrics['MAPE']:.2f}%  "
      f"on {len(y_true)} valid points")

# ── график: история 3 сут + прогноз vs факт ──
# hist_start = pred_idx[0] - pd.Timedelta(hours=1)
hist_start = pred_idx[0] - pd.Timedelta(minutes=30)
plot_history_forecast(
    history  = df.loc[hist_start:pred_idx[0], 'common_delay_p90'],
    forecast = pd.Series(list_of_avg_elements, index=pred_idx),
    actual   = y_true,
    title    = 'Blind forecast vs actual — hold-out 24 h'
)

In [ ]:
# y_true и y_pred уже созданы в предыдущей ячейке

metrics = global_metrics(y_true, list_of_avg_last_15_minutes_elements)
print(f"Blind 24-h test • MAE={metrics['MAE']:.1f}  RMSE={metrics['RMSE']:.1f}  MAPE={metrics['MAPE']:.2f}%  "
      f"on {len(y_true)} valid points")

# ── график: история 3 сут + прогноз vs факт ──
# hist_start = pred_idx[0] - pd.Timedelta(hours=1)
hist_start = pred_idx[0] - pd.Timedelta(minutes=30)
plot_history_forecast(
    history  = df.loc[hist_start:pred_idx[0], 'common_delay_p90'],
    forecast = pd.Series(list_of_avg_last_15_minutes_elements, index=pred_idx),
    actual   = y_true,
    title    = 'Blind forecast vs actual — hold-out 24 h'
)